In [ ]:
import copy

import ipywidgets as widgets
from IPython.display import display

import hyp3_sdk
import asf_search

In [ ]:
hyp3_watermap = hyp3_sdk.HyP3('https://hyp3-watermap.asf.alaska.edu', prompt=True)

In [ ]:
wkt = widgets.Textarea(
    placeholder='WKT of search area',
    value='POLYGON((-91.185 36.6763,-86.825 36.6763,-86.825 38.9176,-91.185 38.9176,-91.185 36.6763))',
    layout=widgets.Layout(width='100%'),
)
display(wkt)

In [ ]:
from asf_search import SENTINEL1, SLC, IW, VV_VH

search_results = asf_search.geo_search(
    platform=[SENTINEL1],
    processingLevel=[SLC],
    beamMode=[IW],
    polarization=[VV_VH],
    intersectsWith=wkt.value,
    start='2021-05-30',
)
all_granules = {result.geojson()['properties']['sceneName'] for result in search_results}

In [ ]:
job_definition = {
    'name': 'water-map-example',
    'job_type': "WATER_MAP",
    'job_parameters': {
        'resolution': 30,
        'speckle_filter': True,
        'max_vv_threshold': -17.0,
        'max_vh_threshold': -24.0,
        'hand_threshold': 15.0,
        'hand_fraction': 0.8,
        'membership_threshold': 0.45,
    }
}

In [ ]:
jobs = hyp3_sdk.Batch()
for granule in all_granules:
    job = copy.deepcopy(job_definition)
    job['job_parameters']['granules'] = [granule]
    jobs += hyp3_watermap.submit_prepared_jobs([job])

In [ ]:
jobs = hyp3_watermap.watch(jobs)

In [ ]:
jobs.download_files('data/')